In [1]:
import requests
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import gymnasium as gym
from stable_baselines3 import DQN

In [2]:
def get_info(response):
    done = response.json().get('done') if not None else False
    info = response.json().get('info')
    infos = response.json().get('info')
    observation = response.json().get('observation') 
    observation = np.array(observation, dtype=np.int32)
    reward = response.json().get('reward')
    trunc = response.json().get('trunc') if not None else False
    if response.status_code != 200:
        print("error code at response ")
    return done,info,observation,reward,trunc
    


new_game = "http://18.185.60.20:5005/new_game"

headers = {
    'Content-Type': 'application/json'
    }

# Start new game
response = requests.post(new_game, headers=headers)
uuid = response.json().get('uuid')


step = "http://18.185.60.20:5005/step/"+uuid
reset = "http://18.185.60.20:5005/reset/"+uuid



In [4]:
response = requests.post(reset, headers=headers)
content = {    'action': 1    }
response = requests.post(step,json=content,headers=headers)

done,info,observation,reward,trunc = get_info(response)

done,info,observation,reward,trunc = get_info(response)
print(info)
print(observation)

{}
[0 0]


In [6]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

class TrainingCallback(BaseCallback):
    def __init__(self, verbose=1):
        super(TrainingCallback, self).__init__(verbose)
        self.episode_rewards = []
        self.episode_lengths = []

    def _on_step(self) -> bool:
        # Assuming that `infos` contains episode information
        # You might need to adjust this depending on your setup
        if 'infos' in self.locals and self.locals['infos']:
            for info in self.locals['infos']:
                if 'episode' in info:
                    episode_info = info['episode']
                    self.episode_rewards.append(episode_info['r'])
                    self.episode_lengths.append(episode_info['l'])
        return True



class MazeAPIEnv(gym.Env):
    def __init__(self, api_step_url, headers,api_reset_url):
        super(MazeAPIEnv, self).__init__()
        
        self.api_step_url = api_step_url  # URL for the API step endpoint
        self.headers = headers  # Headers for authorization or any other required fields
        self.api_reset_url = api_reset_url
        
        # Define the action and observation space
        # Assuming 4 possible actions (up, down, left, right) as an example
        self.action_space = spaces.Discrete(4)
        
        # Observation space could be a position in the 5x5 grid (unknown initially)
        # Here, assuming a flat representation of the maze as 25 positions
        
        # For multi-dimensional observations, use Box space
        self.observation_space = spaces.Box(low=0, high=255, shape=(2,), dtype=np.uint8)

        #self.observation_space = spaces.Discrete(25)
        
        self.current_state = None  # To hold the current state
        self.done = False

    def reset(self,seed=None,**kwargs):
        """Resets the environment by sending a reset request to the API if needed."""
        # If the environment doesn't have a specific reset API, we can assume resetting the agent manually
        # Set the current state to None or starting position
        self.current_state = [0,0]
        self.done = False
        
        response = requests.post(self.api_reset_url, headers=headers)
        if response.status_code !=200 :
            print("erroe code in step")


        return self.current_state, {}
        

    def step(self, action):
        # Send the action to the API
        content = {'action': int(action)}
        response = requests.post(self.api_step_url, json=content, headers=self.headers)
        if response.status_code !=200 :
            print("erroe code in step")
        # Extract the response data
        done,info,observation,reward,truncated = get_info(response)
            
        # Update current state
        self.current_state = observation
        self.done = done
        
        return observation, reward, done, truncated, info


# Define the API endpoint and headers
api_step_url = step  # Replace with actual API endpoint
api_reset_url = reset
headers = {'Content-Type': 'application/json'}

# Instantiate the custom environment
env = MazeAPIEnv(api_step_url, headers,reset)

# Create a DQN agent using the custom environment
model = DQN('MlpPolicy', env, verbose=1)

# Train the agent
# Train the agent

callback = TrainingCallback()
model.learn(total_timesteps=30, callback=callback)


# Save the trained model
model.save("dqn_maze_agent")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [8]:
# Evaluate the trained agent
num_episodes = 10
total_rewards = []

for _ in range(num_episodes):
    obs,_ = env.reset()
    done = False
    episode_reward = 0

    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        episode_reward += reward

    total_rewards.append(episode_reward)

average_reward = sum(total_rewards) / num_episodes
print(f"Average reward over {num_episodes} episodes: {average_reward}")

ConnectTimeout: HTTPConnectionPool(host='18.185.60.20', port=5005): Max retries exceeded with url: /step/10087e9c-4fa9-4b33-9e20-10e5357ca02f (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002206D12D360>, 'Connection to 18.185.60.20 timed out. (connect timeout=None)'))